## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(20)
data.tail(20)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Применила *head(20)*,*tail(20)*:

**days_employed** отрицательные значения и NaN;

**education** используется разный регистр;

**total_income** есть NaN;

**purpose** многие строки повторяются, т.е. одни и те же цели описаны разными словами.

Использую *.info()*:

в таблице 21525 строк и 12 столбцов;

названия столбцов указаны корректно,без ошибок, пробелов, в одном регистре и отображают указанную в них информацию.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data['children'].value_counts(normalize=True)
#поменяла отрицательные значения на положительные
data['children'] = data['children'].apply(abs)

#заменила строки, в которых количество детей равно 20 на медианные значения
data.loc[data['children'] > 19,'children'] = data['children'].median()

#поменяла неопределенный пол на женский
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
#удалила столбец
data.drop(['days_employed'], axis='columns', inplace=True)

#заменила пропуски на медианные значения
# data.loc[data['total_income'].isna(), 'total_income'] = data['total_income'].median()     
data.isna().sum()
#заменила пропуски на медианные значения с группировкой по роду деятельности
for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(income_type, median)
    data.loc[(data['total_income'].isna())&(data['income_type'] == income_type), 'total_income'] = median
data.isna().sum()

сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995


children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Просмотрев столбцы поочередно с помощью *.value_counts(normalize=True)* сделала следующее:

В столбце **children** используем модуль и поменяем отрицательные значения на положительные.Также в этом столбце есть строки с кол-м детей 20(думаю это опечатки),заменим медианными значениями, используя *.loc*.

Меняем в столбце **gender** одно значение 'XNA' на 'F'.

Cтолбец **days_employed** информации нам никакой не дает, в нем есть отрицательные и пропущенные значения, удаляю его методом *.drop()*.

В столбце **dob_years** есть возраст 0, но таких доля таких строк всего 0.004692 от всех строк, считаю что на анализ это никак не повлияет.

Посчитала в каждом столбце пропущенные значения с помощью методов **.isna()** и **.sum()**.
Получила, что в столбце **total_income** 2174 пропуска.Это пропуски NaN, которые возникли случайным образом. Необходимо выполнить замену пропущенных значений. Заменила пропущенные значения с помощью метода *.loc*, на медианные, вычисленные с помощью метода *.median()*.Как я поняла,медианные значения самые универсальные,т.е. в большинстве случаев именно медианные данные будут приближены по максимуму к пропущенным. Выполнила проверку замены пропусков с помощью методов **.isna()** и **.sum()**. Пропусков 0, значит пропущенных данных больше нет.

<font color='purple'>Заменила пропущенные значения в столбце **total_income** на медианные, сгруппированные по роду деятельности, используя метод *.loc* c группировкой по одной переменной.</font>

    

### Замена типа данных

In [3]:
data.dtypes
data['total_income'] = data['total_income'].astype('int')
data['children'] = data['children'].astype('int')
data.dtypes

children             int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод

Вывела на экран типы данных каждого столбца.Использовала *.dtypes*.

Изменила в столбце **total_income** тип с object на int64. Т.к. это доход и выполнять различные действия с ним лучше будет если тип будет целый.

Также столбец **children** после замены пропусков стал float. Изменю на int с помощью **.astype**.

### Обработка дубликатов

In [4]:
data['education'].value_counts()
data['education'] = data['education'].str.lower()
data['education'].value_counts()

data.shape
data = data.drop_duplicates()
data.shape

(21454, 11)

### Вывод

Использовав *.value_counts()* в столбце **education** обнаружила дубликаты-уникальные строки с различным регистром. Думаю они появились при копировании информации из разных файлов. 

Исправила регистр на нижний, использовав *.str.lower()*

<font color='purple'>Удалила дубликаты с помощью метода *.drop_duplicates()*. Было строк 21525. Стало 21454.</font>


<b>UPD <font color=green>Разобрались ✔👍

### Лемматизация

In [5]:
#Загружаем библиотеку
from pymystem3 import Mystem
m = Mystem()
# Выделяем уникальные значения целей
uniq_purpose = data['purpose'].unique()
# Преобразуем список в строку
uniq_purpose = ' '.join(uniq_purpose)
print(uniq_purpose)
# Лемматизируем уникальные значения целей
lemmas_purpose = m.lemmatize(uniq_purpose)
# Находим количество лемм уникальных значений целей
from collections import Counter
print(Counter(lemmas_purpose))



покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием
Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4

### Вывод

Провела лемматизацию для столбца **purpose**. На основе выделенных лемм буду делать далее категоризацию данных.

### Категоризация данных

In [6]:
# Создаю список основных целей
purpose_osn = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба', 'ремонт']
# Создаю лемматизированный столбец целей
data['purpose_lemm'] = data['purpose'].apply(m.lemmatize)
print(data['purpose_lemm'])
#Функция для создания категорий значений целей
def category(purpose_lemm):
     if 'недвижимость' in purpose_lemm or 'жилье'in purpose_lemm:
        return 'недвижимость'
     if 'автомобиль' in purpose_lemm:
        return 'автомобиль'    
     if 'образование' in purpose_lemm:
        return 'образование'
     if 'свадьба' in purpose_lemm:
        return  'свадьба'    
     return 'ошибка'

data['purpose_osn'] = data['purpose_lemm'].apply(category)
data.head(10)

#Выделяю "словарь" для семейного статуса
family = data[['family_status', 'family_status_id']]
#Удаляю дубликаты в словаре
family_drop = family.drop_duplicates().reset_index(drop = True)
family_drop


0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: purpose_lemm, Length: 21454, dtype: object


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Вывод

Сделала категоризацию данных столбца **purpose**. Теперь вместо этого столбца в котором многие строки повторяются, т.е. одни и те же цели описаны разными словами получили столбец **purpose_osn**. В котором у нас указано 4 категории целей.

<font color='purple'>Выделила "словарь" для семейного статуса и удалила дубликаты.</font>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# Делаем сводную таблицу 
children_report = data.pivot_table(index = 'children', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
children_report.columns = ['no_debt', 'debt']
children_report['%'] = children_report['debt'] / (children_report['debt'] + children_report['no_debt'])
children_report.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})  


### Вывод

Люди у которых нет детей возвращают кредит в срок чаще тех у которых дети есть. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_report = data.pivot_table(index = 'family_status', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
family_status_report.columns = ['no_debt', 'debt']
family_status_report['%'] = family_status_report['debt'] / (family_status_report['debt'] + family_status_report['no_debt'])
family_status_report.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

### Вывод

<font color='purple'>Люди в разводе, а также вдовцы/вдовы возвращают кредит в срок чаще других категорий граждан.</font>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Использую метод .qcut для создания групп дохода
data['income_groups'] = pd.qcut(data['total_income'], q=5, 
                        labels=['доход_1', 
                                'доход_2', 
                                'доход_3',  
                                'доход_4', 
                                'доход_5']) 

total_income_report = data.pivot_table(index = 'income_groups', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
total_income_report.columns = ['no_debt', 'debt']
total_income_report['%'] = total_income_report['debt'] / (total_income_report['debt'] + total_income_report['no_debt'])
total_income_report.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})
#data['income_groups'].value_counts()



### Вывод

Верхняя и нижняя группа отдают кредиты чуть лучше средних 

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:

purpose_osn_report = data.pivot_table(index = 'purpose_osn', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
purpose_osn_report.columns = ['no_debt', 'debt']
purpose_osn_report['%'] = purpose_osn_report['debt'] / (purpose_osn_report['debt'] + purpose_osn_report['no_debt'])
purpose_osn_report.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

### Вывод

<font color='purple'>Люди,которые берут кредит на покупку недвижимости и свадьбу возвращают кредит в срок чаще.</font>

### Шаг 4. Общий вывод

<font color='purple'>Для людей в разводе, или вдов/вдовцов, у которых нет детей, и которые хотят взять кредит на покупку недвижимости или свадьбу можно предложить кредит по акции под более низкий процент,т.к. это категории граждан с наименьшими рисками. Тем самым привлечь больше людей и выдать больше кредитов и получить больше прибыли.</font>